In [3]:
import os
import sys
import csv
import requests
import urllib.request
import json
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from textblob import TextBlob
import nltk

sys.path.append('../')
from envir import config
import imp
imp.reload(config)

<module 'envir.config' from '../envir/config.py'>

In [4]:
con = psycopg2.connect(config.con)

In [25]:
engine = create_engine('postgresql+psycopg2:///yelp_abcs', paramstyle="format")
conn = engine.connect()

In [6]:
yelp_busi = pd.read_sql("select * from stag.yelp_academic_business limit 10", con)

In [7]:
yelp_busi.head()

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,"{""GoodForKids"": ""False""}","Golf, Active Life",null
1,1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{""RestaurantsReservations"": ""True"", ""GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{""Monday"": ""9:0-0:0"", ""Tuesday"": ""9:0-0:0"", ""W..."
2,2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{""GoodForKids"": ""True"", ""NoiseLevel"": ""u'avera...","Sushi Bars, Restaurants, Japanese","{""Monday"": ""17:30-21:30"", ""Wednesday"": ""17:30-..."
3,3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,null,"Insurance, Financial Services","{""Monday"": ""8:0-17:0"", ""Tuesday"": ""8:0-17:0"", ..."
4,4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{""BusinessAcceptsBitcoin"": ""False"", ""ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{""Monday"": ""7:0-23:0"", ""Tuesday"": ""7:0-23:0"", ..."


In [8]:
yelp_cities = pd.read_sql('''select city, state, count(*) from stag.yelp_academic_business b
                            
                            WHERE categories ILIKE '%Restaurants%'
                            GROUP BY 1,2''', con)

In [9]:
# will the word toronto/Las vegas be sufficient?
# take a look at the counts of records per city

In [10]:
yelp_cities[yelp_cities['state'] =='NV']

,city,state,count
13,N. Las Vegas,NV,3
20,Sunrise Manor,NV,1
90,Nellis AFB,NV,4
91,Paradise,NV,6
96,N Las Vegas,NV,6
164,Boulder City,NV,51
184,Blue Diamond,NV,3
252,NELLIS AFB,NV,1
265,Clark,NV,1
346,South Las Vegas,NV,2


In [11]:
yelp_cities[yelp_cities['state'] =='ON']

,city,state,count
32,East Ajax,ON,1
38,Centre Island,ON,1
40,Palgrave,ON,1
51,Unionville,ON,26
56,Thornhill,ON,164
58,Oakridges,ON,1
68,AGINCOURT,ON,1
72,Vaughan,ON,474
73,Caledon,ON,20
74,River Drive Park,ON,1


In [ ]:
#takes a few minutes to run

In [3]:
yelp_busi = pd.read_sql('''select * from stag.yelp_academic_business b
                        LEFT JOIN stag.yelp_academic_review r
                        ON b.business_id = r.business_id
                        WHERE b.state IN ('ON', 'NV')
                        AND categories ILIKE '%Restaurants%'
                        ORDER BY b.business_id''', con)

In [15]:
yelp = pd.read_sql('''select * from stag.yelp_academic_business b
                        WHERE b.state IN ('ON', 'NV')
                        AND categories ILIKE '%Restaurants%'
                        ORDER BY b.business_id''', con)

In [1]:
# y_reviews = pd.read_sql('''SELECT r.business_id, r.stars, r.text, r.date from stag.yelp_academic_business b
#                         LEFT JOIN stag.yelp_academic_review r
#                         ON b.business_id = r.business_id
#                         WHERE b.state IN ('ON', 'NV')
#                         AND categories ILIKE '%Restaurants%'
#                         ORDER BY r.business_id''', con)

In [30]:
yelp_busi.head() 

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,index,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,39611,0044Q4cVBcHBgXPmhGgtcQ,The King's Landing,5946 Main Street,Whitchurch-Stouffville,ON,L4A 3A1,43.969338,-79.256383,4.0,...,1819480,2fen-uTpQ6aaX1gS_k6fmw,iDL7gz0iZdIgAELihaujDw,0044Q4cVBcHBgXPmhGgtcQ,4.0,0,1,0,Great place and excellent customer service. Po...,2014-04-07 23:04:06
1,39611,0044Q4cVBcHBgXPmhGgtcQ,The King's Landing,5946 Main Street,Whitchurch-Stouffville,ON,L4A 3A1,43.969338,-79.256383,4.0,...,1333761,hZl88pb5hhWGdxCQwtCoSg,1jsz1T5kDml0O3phegTVhQ,0044Q4cVBcHBgXPmhGgtcQ,5.0,0,0,0,"Small, cute location in Stouffville - friendly...",2016-03-06 23:56:52
2,39611,0044Q4cVBcHBgXPmhGgtcQ,The King's Landing,5946 Main Street,Whitchurch-Stouffville,ON,L4A 3A1,43.969338,-79.256383,4.0,...,1760411,dyr7CD2083GShTFPv-cxbw,7lHoPit-IJWYVMIZPIn78A,0044Q4cVBcHBgXPmhGgtcQ,4.0,0,0,0,So happy to have finally tried this gem in tow...,2018-02-21 17:29:04
3,39611,0044Q4cVBcHBgXPmhGgtcQ,The King's Landing,5946 Main Street,Whitchurch-Stouffville,ON,L4A 3A1,43.969338,-79.256383,4.0,...,1468916,pEaa986ZI5Jyc2HrOT0sOA,0niKeOLbvE5NAVcavzOY7g,0044Q4cVBcHBgXPmhGgtcQ,5.0,0,1,0,"Incredible atmosphere tonight, games on all th...",2017-03-29 02:09:18
4,39611,0044Q4cVBcHBgXPmhGgtcQ,The King's Landing,5946 Main Street,Whitchurch-Stouffville,ON,L4A 3A1,43.969338,-79.256383,4.0,...,1873180,9lLGCMr0InLSmnO75ZNLig,y05H_1uQhO4frLTJ5_tGCQ,0044Q4cVBcHBgXPmhGgtcQ,4.0,0,0,0,Nice place to hang out with friends. Decent pu...,2014-09-21 12:37:38


In [5]:
yelp_busi.columns

Index(['index', 'business_id', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours', 'index', 'review_id',
       'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text',
       'date'],
      dtype='object')

In [2]:
# rename the columns
yelp_busi.columns  = ['index', 'business_id', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'total_stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours', 'index', 'review_id',
       'user_id', 'rev_business_id', 'rating_stars', 'useful', 'funny', 'cool', 'text',
       'date']

NameError: name 'yelp_busi' is not defined

In [60]:
# could run the query above or take a subset of yelp_busi
y_reviews = yelp_busi[['rev_business_id', 'rating_stars', 'text', 'date']]

In [61]:
y_reviews.head()

,rev_business_id,rating_stars,text,date
0,0044Q4cVBcHBgXPmhGgtcQ,4.0,Great place and excellent customer service. Po...,2014-04-07 23:04:06
1,0044Q4cVBcHBgXPmhGgtcQ,5.0,"Small, cute location in Stouffville - friendly...",2016-03-06 23:56:52
2,0044Q4cVBcHBgXPmhGgtcQ,4.0,So happy to have finally tried this gem in tow...,2018-02-21 17:29:04
3,0044Q4cVBcHBgXPmhGgtcQ,5.0,"Incredible atmosphere tonight, games on all th...",2017-03-29 02:09:18
4,0044Q4cVBcHBgXPmhGgtcQ,4.0,Nice place to hang out with friends. Decent pu...,2014-09-21 12:37:38


In [ ]:
#parse the categories

In [33]:
cat = pd.read_sql('''select DISTINCT categories from stag.yelp_academic_business 
''', con)

In [37]:
cat['categories'].apply(lambda x: x.split(', ') if type(x) == str else x).apply(pd.Series).stack().value_counts()

Restaurants                    31076
Shopping                       20905
Food                           18874
Home Services                  11010
Nightlife                      10548
Health & Medical                9915
Beauty & Spas                   9617
Local Services                  9490
Bars                            9282
Event Planning & Services       7474
Active Life                     6721
Automotive                      5654
Fashion                         5325
Professional Services           5312
American (Traditional)          5301
Home & Garden                   5154
Sandwiches                      4848
Arts & Entertainment            4733
Breakfast & Brunch              4195
Specialty Food                  4001
American (New)                  3925
Coffee & Tea                    3771
Doctors                         3693
Hair Salons                     3404
Hair Removal                    3360
Fitness & Instruction           3320
Burgers                         3069
H

In [14]:
cat['categories'].apply(pd.Series).stack().value_counts()

Contractors, Handyman, Home Services, Masonry/Concrete, Stucco Services                                                                                                           1
Pet Services, Pets, Pet Boarding, Pet Sitting, Pet Groomers                                                                                                                       1
Skin Care, Waxing, Nail Salons, Hair Removal, Hair Salons, Beauty & Spas                                                                                                          1
Waxing, Hair Stylists, Hair Removal, Hair Salons, Beauty & Spas                                                                                                                   1
Dentists, Pediatric Dentists, Oral Surgeons, Health & Medical, Orthodontists                                                                                                      1
Photo Booth Rentals, Party Supplies, Party Equipment Rentals, Event Planning & Services             

In [10]:
yelp_busi.categories[9]

"Event Planning & Services, Threading Services, Henna Artists, Fashion, Hair Removal, Shopping, Souvenir Shops, Tattoo, Women's Clothing, Beauty & Spas"

In [16]:
yelp['categories'] = yelp['categories'].apply(lambda x: x.split(', '))

In [17]:
yelp['categories'].head()

0    [Restaurants, Chicken Wings, Canadian (New), B...
1    [Barbeque, Steakhouses, Seafood, Restaurants, ...
2                                [Restaurants, Indian]
3                            [Vietnamese, Restaurants]
4                       [Chinese, Korean, Restaurants]
Name: categories, dtype: object

In [18]:
yelp_df2 = pd.concat([yelp, pd.get_dummies(yelp['categories'].apply(pd.Series).stack()).sum(level=0)], axis=1)

In [19]:
yelp_df2.head()

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,Windshield Installation & Repair,Wine & Spirits,Wine Bars,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,Zoos
0,39611,0044Q4cVBcHBgXPmhGgtcQ,The King's Landing,5946 Main Street,Whitchurch-Stouffville,ON,L4A 3A1,43.969338,-79.256383,4.0,...,0,0,0,0,0,0,0,0,0,0
1,46955,005XmZKuJZuNbl5tGXc5SA,Tony Roma's,620 E Sahara Ave,Las Vegas,NV,89104,36.144308,-115.145691,3.0,...,0,0,0,0,0,0,0,0,0,0
2,85575,00BHqa91AXz20Ca5k-Peiw,Dosa Darbar,1150 Albion Rd,Etobicoke,ON,M9V 1A8,43.737948,-79.567273,2.0,...,0,0,0,0,0,0,0,0,0,0
3,17823,00idQeDRVoL8WN4J7TnEJQ,A Taste of Pho Vietnamese Cuisine,4350 Steeles Avenue E,Markham,ON,L3R 9V4,43.825447,-79.305043,3.5,...,0,0,0,0,0,0,0,0,0,0
4,176089,00I_YBjgZAvd2ggZr3J0cQ,Shanghai Chinese Restaurant,5451 Yonge Street,Toronto,ON,M2N 5S1,43.777360,-79.414539,3.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# check the output
yelp_df2[['categories', 'Restaurants', 'Chicken Wings', 'Canadian (New)']].head()

,categories,Restaurants,Chicken Wings,Canadian (New)
0,"[Restaurants, Chicken Wings, Canadian (New), B...",1,1,1
1,"[Barbeque, Steakhouses, Seafood, Restaurants, ...",1,0,0
2,"[Restaurants, Indian]",1,0,0
3,"[Vietnamese, Restaurants]",1,0,0
4,"[Chinese, Korean, Restaurants]",1,0,0


In [88]:
# some weird ones?
yelp['categories'].apply(pd.Series).stack().value_counts()

Restaurants                  22614
Food                          5676
Nightlife                     2854
Bars                          2708
Fast Food                     2489
Chinese                       2208
Sandwiches                    2179
Pizza                         2051
American (Traditional)        2000
Burgers                       1819
Breakfast & Brunch            1816
Italian                       1745
Mexican                       1441
Japanese                      1431
Coffee & Tea                  1365
Cafes                         1345
American (New)                1219
Canadian (New)                1147
Seafood                       1027
Sushi Bars                     994
Chicken Wings                  975
Event Planning & Services      953
Specialty Food                 936
Asian Fusion                   935
Indian                         820
Mediterranean                  791
Desserts                       758
Thai                           743
Middle Eastern      

In [27]:
yelp_df2.to_sql('business_dummied_categories', engine, schema ='clean', if_exists='replace')

In [ ]:
#descriptive analysis of date

In [55]:
yelp_busi['date'] = pd.to_datetime(yelp_busi['date'])

In [58]:
yelp_busi[yelp_busi['city']=='Las Vegas']['date'].describe()

count                 1242697
unique                1238076
top       2014-06-19 18:04:15
freq                        5
first     2005-01-24 22:21:29
last      2018-11-14 18:12:40
Name: date, dtype: object

In [57]:
yelp_busi[yelp_busi['city']=='Toronto']['date'].describe()

count                  376576
unique                 375867
top       2014-07-27 01:48:29
freq                        5
first     2007-12-15 00:38:57
last      2018-11-14 18:04:51
Name: date, dtype: object

In [92]:
yelp_busi['business_id'].nunique()

22614

In [90]:
yelp.shape

(22614, 15)

In [ ]:
# testing sentiment polarity

In [22]:
yelp_busi['text'][10]

'We found this place accidentally last year. When my boss asked to meet with me midway today my husband actually remembered the name and asked to come. The place is open, modern and clean. Quite attractive and comfortable. The food is really  good and the service is excellent. Last time we were there we both enjoyed our meal but kept watching the dishes other people were ordering and ended up making a take out order to bring home for supper. My boss enjoyed her food and was pleased the place. It was very busy and sometimes the noise level was a bit high, but well worth having to ask someone to repeat themselves.'

In [47]:
yelp_busi['text'][100]

'This is not our usual Tony\'s but we were on this side of town so we tried it out. The food was good. We both had steak and they came out like we like them. My drink on the other hand was not great. The bartender left for the day & my waiter..who was very SLOW had to make it. Eghh. He was annoying the crap out of my hubby with his lack of concern for us. We never got the bill so we "guessed" what what owed and dropped cash down & left. Wont be coming back to this one again.'

In [48]:
TextBlob(yelp_busi['text'][100]).sentiment.polarity

-0.2018686868686869

In [ ]:
#apply sentiment analysis

In [93]:
yelp_busi['sentiment'] = yelp_busi['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
#returns the sentiment of text
#by returning a value between -1.0 and 1.0
#takes a LONG TIME! 

In [101]:
review_sentiment = yelp_busi[['business_id', 'review_id', 'rating_stars', 'sentiment', 'date']]

In [103]:
review_sentiment.head()

,business_id,review_id,rating_stars,sentiment,date
0,0044Q4cVBcHBgXPmhGgtcQ,2fen-uTpQ6aaX1gS_k6fmw,4.0,0.534722,2014-04-07 23:04:06
1,0044Q4cVBcHBgXPmhGgtcQ,hZl88pb5hhWGdxCQwtCoSg,5.0,0.213112,2016-03-06 23:56:52
2,0044Q4cVBcHBgXPmhGgtcQ,dyr7CD2083GShTFPv-cxbw,4.0,0.515741,2018-02-21 17:29:04
3,0044Q4cVBcHBgXPmhGgtcQ,pEaa986ZI5Jyc2HrOT0sOA,5.0,0.581212,2017-03-29 02:09:18
4,0044Q4cVBcHBgXPmhGgtcQ,9lLGCMr0InLSmnO75ZNLig,4.0,0.195238,2014-09-21 12:37:38


In [110]:
review_sentiment.to_sql('review_sentiment', engine, schema ='clean')

In [ ]:
#test that it worked 

In [13]:
pd.read_sql('SELECT * FROM clean.review_sentiment LIMIT 10', conn)

,index,business_id,review_id,rating_stars,sentiment,date
0,0,0044Q4cVBcHBgXPmhGgtcQ,2fen-uTpQ6aaX1gS_k6fmw,4.0,0.534722,2014-04-07 23:04:06
1,1,0044Q4cVBcHBgXPmhGgtcQ,hZl88pb5hhWGdxCQwtCoSg,5.0,0.213112,2016-03-06 23:56:52
2,2,0044Q4cVBcHBgXPmhGgtcQ,dyr7CD2083GShTFPv-cxbw,4.0,0.515741,2018-02-21 17:29:04
3,3,0044Q4cVBcHBgXPmhGgtcQ,pEaa986ZI5Jyc2HrOT0sOA,5.0,0.581212,2017-03-29 02:09:18
4,4,0044Q4cVBcHBgXPmhGgtcQ,9lLGCMr0InLSmnO75ZNLig,4.0,0.195238,2014-09-21 12:37:38
5,5,0044Q4cVBcHBgXPmhGgtcQ,iAlYQhJ5FyHr0EdmHzgdnw,5.0,0.505556,2014-09-09 22:16:11
6,6,0044Q4cVBcHBgXPmhGgtcQ,23_iCAndaciqgQoiyewqfw,5.0,0.235714,2016-07-14 03:29:57
7,7,0044Q4cVBcHBgXPmhGgtcQ,6ua2-9SQEF7tWIjRtgY4EA,5.0,0.235714,2016-08-15 22:53:36
8,8,0044Q4cVBcHBgXPmhGgtcQ,rvn8R2ix2y-pCoZBuY2DSA,5.0,0.418889,2018-03-13 16:37:22
9,9,0044Q4cVBcHBgXPmhGgtcQ,6PLXxLB85FPVWMxhGfx9dA,5.0,0.318750,2015-07-16 17:12:01


In [21]:
pd.read_sql('SELECT count(*) FROM clean.review_sentiment', conn)

,count
0,1965918


In [ ]:
#look at how to parse the attributes

In [18]:
s = yelp_busi['attributes'][1]
d = json.loads(s)

In [19]:
d

{'Alcohol': "u'full_bar'",
 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}",
 'BikeParking': 'True',
 'BusinessAcceptsCreditCards': 'True',
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
 'Caters': 'True',
 'DriveThru': 'False',
 'GoodForKids': 'True',
 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
 'HasTV': 'True',
 'NoiseLevel': "u'average'",
 'OutdoorSeating': 'False',
 'RestaurantsAttire': "u'casual'",
 'RestaurantsDelivery': 'False',
 'RestaurantsGoodForGroups': 'True',
 'RestaurantsPriceRange2': '2',
 'RestaurantsReservations': 'True',
 'RestaurantsTableService': 'True',
 'RestaurantsTakeOut': 'True',
 'WiFi': "'no'"}

In [11]:
yelp_busi.attributes[9]

'{"BikeParking": "True", "BusinessParking": "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", "BusinessAcceptsCreditCards": "True", "BusinessAcceptsBitcoin": "False", "ByAppointmentOnly": "False", "WheelchairAccessible": "True", "RestaurantsPriceRange2": "2"}'

In [ ]:
#location of work
#/green-projects/project-nyc_food_health/workspace/sjf374/yelp_abcs

In [ ]:
#taking a look at the database and access

In [29]:
roles = pd.read_sql("select * from pg_roles", con)

In [30]:
mem = pd.read_sql("select * from pg_auth_members", con)

In [8]:
tables = pd.read_sql("select * from pg_tables WHERE schemaname = 'stag'", con)

In [9]:
tables

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers
0,stag,ny_zipcodes,yg833,None,False,False,False
1,stag,yelp_business_type_totals,yg833,None,False,False,False
2,stag,yelp_businesses,yg833,None,False,False,False


In [ ]:
m.member = cte.oid

In [34]:
mem[mem['member'] == 1764017]

,roleid,member,grantor,admin_option
170,1764162,1764017,10,False
174,1764163,1764017,10,False
177,1764164,1764017,10,False


In [32]:
roles[roles['rolname'] == 'sjf374']

,rolname,rolsuper,rolinherit,rolcreaterole,rolcreatedb,rolcatupdate,rolcanlogin,rolreplication,rolconnlimit,rolpassword,rolvaliduntil,rolconfig,oid
166,sjf374,False,True,False,False,False,True,False,-1,********,None,None,1764017


In [22]:
roles[roles['rolname'] == 'yelp_abcs_admin']

,rolname,rolsuper,rolinherit,rolcreaterole,rolcreatedb,rolcatupdate,rolcanlogin,rolreplication,rolconnlimit,rolpassword,rolvaliduntil,rolconfig,oid
529,yelp_abcs_admin,False,True,False,False,False,False,False,-1,********,None,None,1764164
